# Commands

In [ ]:
# cd ~/twinews-logs ; jupython -o nohup-bm25-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/bm25.ipynb

# Init

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # isNotebook, True, False

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from twinews.utils import *
from twinews.models.ranking import *

In [ ]:
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
from gensim.summarization import bm25

In [ ]:
logger = Logger(tmpDir('logs') + "/bm25.log") if isNotebook else Logger("bm25-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

# Config

In [ ]:
config = \
{
    'splitVersion': 2,
    
    'maxUsers': 2 if TEST else None, # Sub-sampling
    'minDF': None if TEST else 1 / 2000, # 1 / 2000 or None
    'maxDF': None if TEST else 300, # 300 or None
    
    'lowercase': False if TEST else False,
    'doLemmatization': False if TEST else False,
    
    'k1': 1.5,
    'b': 0.75,
    'epsilon': 0.25,
}

In [ ]:
modelName = 'bm25'

In [ ]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

# Getting data

In [ ]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

In [ ]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [ ]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

In [ ]:
# We get sentences:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger)

In [ ]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

In [ ]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

In [ ]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

In [ ]:
# Filtering the corpus:
if config['minDF'] is not None or config['maxDF'] is not None:
    docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                        removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
    for doc in docs: assert len(doc) > 0
    bp(docs, logger)

In [ ]:
tt.tic("Data preprocessed")

# Ranking

In [ ]:
urlDocs = dict()
for i in range(len(newsList)):
    urlDocs[newsList[i]] = docs[i]

In [ ]:
rankings = dict()
pbar = ProgressBar(len(evalData['candidates']), logger=logger, message="Generating rankings", printRatio=0.01)
for userId in evalData['candidates']:
    request = []
    for url in evalData['trainUsers'][userId]:
        request += urlDocs[url]
    currentRankings = []
    for candidates in evalData['candidates'][userId]:
        candidates = list(candidates)
        urlCorpus = dict()
        for i in range(len(candidates)):
            urlCorpus[candidates[i]] = urlDocs[candidates[i]]
        urlCorpus = list(urlCorpus.items())
        corpus = [e[1] for e in urlCorpus]
        try:
            # This is the new version not yet pushed on pipy:
            model = bm25.BM25(corpus, k1=config['k1'], b=config['b'], epsilon=config['epsilon'])
        except:
            # This is the actual version with parameters as global variables:
            bm25.PARAM_K1 = config['k1']
            bm25.PARAM_B = config['b']
            bm25.EPSILON = config['epsilon']
            model = bm25.BM25(corpus)
        scores = model.get_scores(request)
        scoresWithUrl = []
        for i in range(len(scores)):
            scoresWithUrl.append((urlCorpus[i][0], scores[i]))
        ranking = sortBy(scoresWithUrl, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        currentRankings.append(ranking)
    rankings[userId] = currentRankings
    pbar.tic(str(userId))

In [ ]:
bp(rankings, logger, 4)

In [ ]:
tt.tic("Rankings done")

# Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())